In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from scipy.spatial import ConvexHull
from mpl_toolkits.mplot3d import Axes3D

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.multioutput import MultiOutputRegressor
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr

# Data Processing

In [39]:
Data = pd.read_csv('SCNP_dataset.csv')
Data

,Comp,BID,SID,CID,rg,asph,acyl,anis,num dom,seg,...,eig2,eig3,eig4,eig5,eig6,eig7,eig8,eig9,eig10,n_eig
0,0.1,0.2,0,0,12.228701,61.184513,30.778156,0.212441,2,48,...,0.001016,0.001885,0.004115,0.004368,0.007120,0.008917,0.010192,0.010418,0.012773,0.000021
1,0.1,0.2,0,1,10.906473,45.301393,22.153887,0.183696,1,25,...,0.001981,0.002270,0.003840,0.004823,0.006086,0.007630,0.008078,0.008877,0.012746,0.000039
2,0.1,0.2,0,2,11.597649,51.393123,27.237148,0.188593,2,36,...,0.001227,0.001769,0.003387,0.004120,0.005110,0.005742,0.007127,0.008390,0.012078,0.000030
3,0.1,0.2,0,3,16.041229,155.108298,63.086271,0.407451,4,105,...,0.000499,0.002372,0.003987,0.005094,0.005925,0.007164,0.009113,0.011217,0.012627,0.000008
4,0.1,0.2,0,4,15.312193,139.605066,62.681879,0.410806,6,108,...,0.000487,0.002857,0.003377,0.005003,0.006256,0.006961,0.009035,0.012391,0.014306,0.000008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7675,0.8,0.8,39,19,7.353935,17.969779,10.097279,0.143275,1,4,...,0.005110,0.012039,0.013861,0.024233,0.024647,0.028329,0.041535,0.043056,0.048735,0.000017
7676,0.8,0.8,39,20,7.110865,18.012399,10.576299,0.166412,1,4,...,0.007475,0.012283,0.014657,0.019058,0.027062,0.029793,0.038675,0.041851,0.047963,0.000016
7677,0.8,0.8,39,21,7.068580,12.873496,6.076033,0.082558,1,4,...,0.004853,0.009125,0.019311,0.021287,0.026657,0.030698,0.034977,0.036610,0.048004,0.000028
7678,0.8,0.8,39,22,7.411105,21.967894,12.709280,0.208238,1,4,...,0.005929,0.011884,0.012992,0.019262,0.021154,0.028907,0.034192,0.040488,0.044181,0.000021


In [40]:
#Extract input data
Input = Data[['Comp', 'BID', 'SID']].drop_duplicates()
with open("SCNP_sequences.txt", "r", encoding="utf-8") as file:
    Sequence = file.readlines()  # Each line becomes a list element
Input['Sequence'] = Sequence
Input

,Comp,BID,SID,Sequence
0,0.1,0.2,0,0000000000000000000001000000000000010001000000...
24,0.1,0.2,1,0011100000000000000000001100000100100010000000...
48,0.1,0.2,2,0000000000000001000000001010000000110000000000...
72,0.1,0.2,3,0001000000000000000000000000010000010000001000...
96,0.1,0.2,4,0000000000000000000000000000000000000000000001...
...,...,...,...,...
7560,0.8,0.8,35,1111111111111111111111111111111111111111111111...
7584,0.8,0.8,36,0001111111111111111111111110011111111111111111...
7608,0.8,0.8,37,1111111111111100000001111111011111111111111111...
7632,0.8,0.8,38,0000001111111111111100000111111111100011111111...


In [41]:
Data = Data.merge(Input, on=['Comp', 'BID', 'SID'], how = 'left')
Data

,Comp,BID,SID,CID,rg,asph,acyl,anis,num dom,seg,...,eig3,eig4,eig5,eig6,eig7,eig8,eig9,eig10,n_eig,Sequence
0,0.1,0.2,0,0,12.228701,61.184513,30.778156,0.212441,2,48,...,0.001885,0.004115,0.004368,0.007120,0.008917,0.010192,0.010418,0.012773,0.000021,0000000000000000000001000000000000010001000000...
1,0.1,0.2,0,1,10.906473,45.301393,22.153887,0.183696,1,25,...,0.002270,0.003840,0.004823,0.006086,0.007630,0.008078,0.008877,0.012746,0.000039,0000000000000000000001000000000000010001000000...
2,0.1,0.2,0,2,11.597649,51.393123,27.237148,0.188593,2,36,...,0.001769,0.003387,0.004120,0.005110,0.005742,0.007127,0.008390,0.012078,0.000030,0000000000000000000001000000000000010001000000...
3,0.1,0.2,0,3,16.041229,155.108298,63.086271,0.407451,4,105,...,0.002372,0.003987,0.005094,0.005925,0.007164,0.009113,0.011217,0.012627,0.000008,0000000000000000000001000000000000010001000000...
4,0.1,0.2,0,4,15.312193,139.605066,62.681879,0.410806,6,108,...,0.002857,0.003377,0.005003,0.006256,0.006961,0.009035,0.012391,0.014306,0.000008,0000000000000000000001000000000000010001000000...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7675,0.8,0.8,39,19,7.353935,17.969779,10.097279,0.143275,1,4,...,0.012039,0.013861,0.024233,0.024647,0.028329,0.041535,0.043056,0.048735,0.000017,0000111111111111100001111111110000001111111111...
7676,0.8,0.8,39,20,7.110865,18.012399,10.576299,0.166412,1,4,...,0.012283,0.014657,0.019058,0.027062,0.029793,0.038675,0.041851,0.047963,0.000016,0000111111111111100001111111110000001111111111...
7677,0.8,0.8,39,21,7.068580,12.873496,6.076033,0.082558,1,4,...,0.009125,0.019311,0.021287,0.026657,0.030698,0.034977,0.036610,0.048004,0.000028,0000111111111111100001111111110000001111111111...
7678,0.8,0.8,39,22,7.411105,21.967894,12.709280,0.208238,1,4,...,0.011884,0.012992,0.019262,0.021154,0.028907,0.034192,0.040488,0.044181,0.000021,0000111111111111100001111111110000001111111111...


# Principal Component Analysis (PCA)

We take advantage of PCA as the dimensionality technique to simplify the sequencing. PCA projects data into a new coordinate system where the firts few coordinates (principal components) capture the most variance in data. The basica goal of PCA is to transform the original features into a smaller set of uncorrelated features while retaining as much information (variance) as possible.

The PCA process is as follows:

1/ **Standardize the Data:** Scale the data for PCA. The data is *mean-centered* (subtract the mean of each feature) and *scaled* (divided by the standard deviation) so that each feature has zero mean and unit variance.

2/ **Compute the Covariance Matrix:** The covariance matrix measures how much the features vary with respect to each other. It helps us understand the relationships between different features.

3/ **Eigenvalue Decomposition:** The covariance matrix is decomposed into eigenvectors and eigenvalues. The eigenvectors represent the principal components (directions of maximum variance), and the eigenvalues show how much variance each principal component accounts for.

4/ **Sort Eigenvalues and Eigenvectors:** Sort the eigenvectors based on the eigenvalues, from largest to smallest. The larger the eigenvalue, the more variance that principal component accounts for.

5/ **Projection onto New Axes:** The data is projected onto the eigenvectors (principal components). This results in a new set of features (principal components) that are linear combinations of the original features. The first principal component captures the largest variance, the second one captures the second largest, and so on.

6/ **Dimensionality Reduction:** By selecting the top *k* eigenvectors (those with the largest eigenvalues), we reduce the dataset’s dimensions. You can control how much variance is retained by adjusting k (the number of principal components you choose).


***We use a 70/15/15 split for the Train/Validate/Test sets***

In [ ]:
unique_sequences = Data["Sequence"].unique()
# Ensure a proper 70/15/15 split (Train/Validation/Test)
train_seq, temp_seq = train_test_split(unique_sequences, test_size=0.3, random_state=42)
val_seq, test_seq = train_test_split(temp_seq, test_size=0.5, random_state=42)  # Split remaining 30% into 15%/15%

# Assign each row to the corresponding split based on its Sequence
df_train = Data[Data["Sequence"].isin(train_seq)].copy()
df_val = Data[Data["Sequence"].isin(val_seq)].copy()
df_test = Data[Data["Sequence"].isin(test_seq)].copy()

# Convert Sequence column (binary string) to numerical array
def sequence_to_array(seq):
    seq = str(seq).strip()  # Remove spaces/newlines
    seq = ''.join(filter(lambda x: x in '01', seq))  # Keep only '0' and '1'
    return list(map(int, list(seq)))  # Convert to list of integers

df_train["Sequence"] = df_train["Sequence"].apply(sequence_to_array)
df_val["Sequence"] = df_val["Sequence"].apply(sequence_to_array)
df_test["Sequence"] = df_test["Sequence"].apply(sequence_to_array)

# Ensure all sequences have the same length
seq_length = len(df_train["Sequence"].iloc[0])
assert all(len(seq) == seq_length for seq in df_train["Sequence"]), "Sequence length mismatch!"

# Convert sequences to NumPy arrays
X_seq_train = np.vstack(df_train["Sequence"].values)
X_seq_val = np.vstack(df_val["Sequence"].values)
X_seq_test = np.vstack(df_test["Sequence"].values)

# Apply PCA (reduce to 10 principal components)
pca = PCA(n_components=10)
X_seq_train_pca = pca.fit_transform(X_seq_train)
X_seq_val_pca = pca.transform(X_seq_val)
X_seq_test_pca = pca.transform(X_seq_test)

# Add PCA-transformed sequences back to DataFrame
for i in range(10):
    df_train[f"PCA_{i}"] = X_seq_train_pca[:, i]
    df_val[f"PCA_{i}"] = X_seq_val_pca[:, i]
    df_test[f"PCA_{i}"] = X_seq_test_pca[:, i]

# Drop original Sequence column (no longer needed)
df_train = df_train.drop(columns=["Sequence"])
df_val = df_val.drop(columns=["Sequence"])
df_test = df_test.drop(columns=["Sequence"])

# Save preprocessed data
df_train.to_csv("train_data_pca.csv", index=False)
df_val.to_csv("val_data_pca.csv", index=False)
df_test.to_csv("test_data_pca.csv", index=False)

# Print results
print(f"Train: {len(train_seq)} sequences, {df_train.shape[0]} total samples")
print(f"Validation: {len(val_seq)} sequences, {df_val.shape[0]} total samples")
print(f"Test: {len(test_seq)} sequences, {df_test.shape[0]} total samples")

Train: 224 sequences, 5376 total samples
Validation: 48 sequences, 1152 total samples
Test: 48 sequences, 1152 total samples


## Gaussian Process Regression (GPR)

GPR is a non-parametric method for regression tasks. It's based on the **Gaussian Process** (GP), which are collections of random variables, any finite number of which have a joint Gaussian distribution. GPR provides a distribution over possible functions that fit the data and can predict both the mean and the uncertainty (variance) of the predictions.

1/ **Prior Distributions:** In GPR, we define a prior distribution over functions. A Gaussian process is specified by a **mean function** (often assumed to be zero) and a **covariance function** (kernel) that defines the correlation between pairs of data points.

2/ **Kernel Function:** The kernel defines the covariamce between two points. A commonly used kernel is te **Radial Basis Function (RBF)**, also known as the **Gaussian kernel**, which measures similarity between points based on their distance. It controls how "smooth" the function is.
$$k(x,x') = exp(-\frac{|x-x'|^2}{2\sigma^2}))$$ 
where $\sigma$ is a length parameter.

3/ **Likelihood Function:** The likelihood describes how the observed data relates to the underlying function. Typically, a Gaussian noise model is assumed, meaning that the data is generated from the underlying coupled with some normally distributed noise.

4/ **Posterior Distribution:** After observing the data, we update our prior belief (distribution over functions) to a **posterior** distribution. The posterior is also a Gaussian process, and it is conditioned on the observed data points.

5/ **Predictions:** GPR not only predicts the mean of the function at new points, but also provides the uncertainty (variance) of the prediction.

In [ ]:
# Define the kernel (RBF kernel with length scale of 1.0, ConstantKernel for scaling)
kernel = C(1.0, (1e-4, 1e1)) * RBF(1.0, (1e-4, 1e1))

#Use BFGS as optimization solver
# Initialize the Gaussian Process Regressor wrapped in MultiOutputRegressor
gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, optimizer='fmin_l_bfgs_b', alpha=1e-6)

# Training data
X_train = df_train[['Comp', 'BID', 'PCA_0', 'PCA_1', 'PCA_2', 'PCA_3', 'PCA_4', 'PCA_5', 'PCA_6', 'PCA_7', 'PCA_8', 'PCA_9']]
y_train = df_train[['rg', 'asph', 'acyl', 'anis']]

# Fit the model
gpr.fit(X_train, y_train)

# Validation set
X_val = df_val[['Comp', 'BID', 'PCA_0', 'PCA_1', 'PCA_2', 'PCA_3', 'PCA_4', 'PCA_5', 'PCA_6', 'PCA_7', 'PCA_8', 'PCA_9']]
y_val = df_val[['rg', 'asph', 'acyl', 'anis']]

# Predict on the validation set
y_val_pred = gpr.predict(X_val)

# Evaluate using Mean Squared Error (MSE) for each target
mse = mean_squared_error(y_val, y_val_pred, multioutput='raw_values')
print(f"Validation MSE for each output: {mse}")

# Compute overall average MSE
mean_mse = mse.mean()
print(f"Overall Validation MSE: {mean_mse}")

c:\Users\pt3594\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\gaussian_process\kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


Validation MSE for each output: [6.88622079e+00 1.69300935e+03 2.48540613e+02 1.57589908e-02]
Overall Validation MSE: 487.1129854734017


In [ ]:
# Initialize the Gaussian Process Regressor wrapped in MultiOutputRegressor
kernel = C(1.0, (1e-4, 1e2)) * RBF(1.0, (1e-4, 1e2))  # Increase upper bound from 10 to 100

# Training data
X_train = df_train[['Comp', 'BID', 'PCA_0', 'PCA_1', 'PCA_2', 'PCA_3', 'PCA_4', 'PCA_5', 'PCA_6', 'PCA_7', 'PCA_8', 'PCA_9']]
y_train = df_train[['rg', 'asph', 'acyl', 'anis']]

# Fit the model
gpr.fit(X_train, y_train)

# Validation set
X_val = df_val[['Comp', 'BID', 'PCA_0', 'PCA_1', 'PCA_2', 'PCA_3', 'PCA_4', 'PCA_5', 'PCA_6', 'PCA_7', 'PCA_8', 'PCA_9']]
y_val = df_val[['rg', 'asph', 'acyl', 'anis']]

# Predict on the validation set
y_val_pred = gpr.predict(X_val)

# Evaluate using Mean Squared Error (MSE) for each target
mse = mean_squared_error(y_val, y_val_pred, multioutput='raw_values')
print(f"Validation MSE for each output: {mse}")

# Compute overall average MSE
mean_mse = mse.mean()
print(f"Overall Validation MSE: {mean_mse}")

c:\Users\pt3594\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\gaussian_process\kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


Validation MSE for each output: [6.88622079e+00 1.69300935e+03 2.48540613e+02 1.57589908e-02]
Overall Validation MSE: 487.1129854734017


Separate Kernels for each properties

In [20]:
from sklearn.preprocessing import StandardScaler
# Standardize the inputs and outputs
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_scaled = scaler_X.fit_transform(df_train[['Comp', 'BID', 'PCA_0', 'PCA_1', 'PCA_2', 'PCA_3', 'PCA_4', 'PCA_5', 'PCA_6', 'PCA_7', 'PCA_8', 'PCA_9']])
X_val_scaled = scaler_X.transform(df_val[['Comp', 'BID', 'PCA_0', 'PCA_1', 'PCA_2', 'PCA_3', 'PCA_4', 'PCA_5', 'PCA_6', 'PCA_7', 'PCA_8', 'PCA_9']])

# Target variables
y_train = df_train[['rg', 'asph', 'acyl', 'anis']]
y_val = df_val[['rg', 'asph', 'acyl', 'anis']]

# Scale each target separately
scaler_y_list = []
y_train_scaled_list = []
y_val_scaled_list = []

for i in range(y_train.shape[1]):
    scaler = StandardScaler()
    y_train_scaled = scaler.fit_transform(y_train.iloc[:, i].values.reshape(-1, 1))
    y_val_scaled = scaler.transform(y_val.iloc[:, i].values.reshape(-1, 1))
    
    scaler_y_list.append(scaler)
    y_train_scaled_list.append(y_train_scaled)
    y_val_scaled_list.append(y_val_scaled)

# Define different kernels for each output
kernels = [
    C(1.0, (1e-4, 1e3)) * RBF(1.0, (1e-4, 1e3)),  # rg
    C(1.0, (1e-4, 1e3)) * RBF(1.0, (1e-4, 1e3)),  # asph
    C(1.0, (1e-4, 1e5)) * RBF(10.0, (1e-4, 1e5)), # acyl (larger kernel)
    C(1.0, (1e-4, 1e3)) * RBF(1.0, (1e-4, 1e3))   # anis
]

# Train separate GPR models
gpr_models = []
y_val_preds = []

for i in range(4):
    gpr = GaussianProcessRegressor(kernel=kernels[i], n_restarts_optimizer=5, alpha=1e-6)
    gpr.fit(X_train_scaled, y_train_scaled_list[i].ravel())  # Flatten y_train
    
    y_val_pred_scaled = gpr.predict(X_val_scaled)
    y_val_pred = scaler_y_list[i].inverse_transform(y_val_pred_scaled.reshape(-1, 1))  # Rescale
    
    gpr_models.append(gpr)
    y_val_preds.append(y_val_pred)

# Convert predictions back to DataFrame
y_val_pred_df = pd.DataFrame(np.hstack(y_val_preds), columns=['rg', 'asph', 'acyl', 'anis'])

# Evaluate using Mean Squared Error (MSE) for each target
mse = mean_squared_error(y_val, y_val_pred_df, multioutput='raw_values')
print(f"Validation MSE for each output: {mse}")

# Compute overall average MSE
mean_mse = mse.mean()
print(f"Overall Validation MSE: {mean_mse}")

c:\Users\pt3594\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\gaussian_process\_gpr.py:663: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
c:\Users\pt3594\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\gaussian_process\kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 0.0001. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


Validation MSE for each output: [3.76684079e+00 1.50362161e+03 3.64591474e+02 1.37174597e-02]
Overall Validation MSE: 467.9984099687234


In [43]:
# Assuming df_train, df_val, df_test already have the structure with PCA applied

# Rescale the features (input features)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(df_train[['Comp', 'BID', 'PCA_0', 'PCA_1', 'PCA_2', 'PCA_3', 'PCA_4', 'PCA_5', 'PCA_6', 'PCA_7', 'PCA_8', 'PCA_9']])
X_val_scaled = scaler.transform(df_val[['Comp', 'BID', 'PCA_0', 'PCA_1', 'PCA_2', 'PCA_3', 'PCA_4', 'PCA_5', 'PCA_6', 'PCA_7', 'PCA_8', 'PCA_9']])
X_test_scaled = scaler.transform(df_test[['Comp', 'BID', 'PCA_0', 'PCA_1', 'PCA_2', 'PCA_3', 'PCA_4', 'PCA_5', 'PCA_6', 'PCA_7', 'PCA_8', 'PCA_9']])

# Define kernels for each output variable
kernels = {
    "rg": C(1.0, (1e-4, 1e3)) * RBF(1.0, (1e-4, 1e3)),
    "asph": C(1.0, (1e-4, 1e3)) * RBF(1.0, (1e-4, 1e3)),
    "acyl": C(1.0, (1e-4, 1e5)) * RBF(10.0, (1e-4, 1e5)),  # Different scaling
    "anis": C(1.0, (1e-4, 1e3)) * RBF(1.0, (1e-4, 1e3))
}

# Initialize a dictionary to store the models and predictions
gpr_models = {}
y_pred_val = {}

# Train separate GPR models for each output variable
for output_var in ["rg", "asph", "acyl", "anis"]:
    print(f"Training GPR for {output_var}...")

    # Extract the corresponding target variable
    y_train = df_train[output_var].values
    y_val = df_val[output_var].values
    
    # Train the Gaussian Process Regressor for this output variable
    gpr = GaussianProcessRegressor(kernel=kernels[output_var], n_restarts_optimizer=10, alpha=1e-6)
    gpr.fit(X_train_scaled, y_train)

    # Store the trained model
    gpr_models[output_var] = gpr

    # Predict on the validation set
    y_pred_val[output_var] = gpr.predict(X_val_scaled)

# Evaluate the models using Mean Squared Error (MSE)
mse_vals = {var: mean_squared_error(df_val[var], y_pred_val[var]) for var in ["rg", "asph", "acyl", "anis"]}
overall_mse = np.mean(list(mse_vals.values()))

# Print the results
print(f"Validation MSE for each output: {mse_vals}")
print(f"Overall Validation MSE: {overall_mse}")

# Optionally, you can check the shape of the predictions to ensure you have 24 outputs per sequence
print(f"Shape of predictions: {y_pred_val['rg'].shape}")


Training GPR for rg...


c:\Users\pt3594\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\gaussian_process\_gpr.py:663: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


Training GPR for asph...


c:\Users\pt3594\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\gaussian_process\_gpr.py:663: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
c:\Users\pt3594\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\gaussian_process\kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 1000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\pt3594\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\gaussian_process\kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 0.0001. Decreasing the bound and ca

Training GPR for acyl...


c:\Users\pt3594\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\gaussian_process\_gpr.py:663: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


Training GPR for anis...
Validation MSE for each output: {'rg': 3.790251263900411, 'asph': 5758.471426985212, 'acyl': 1027.543636222164, 'anis': 0.014459213515279476}
Overall Validation MSE: 1697.4549434211979
Shape of predictions: (1152,)


In [ ]:
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, Matern
# Scale target variables
scaler_Y = {}
y_train_scaled = {}
y_val_scaled = {}

for output_var in ["rg", "asph", "acyl", "anis"]:
    scaler_Y[output_var] = StandardScaler()
    y_train_scaled[output_var] = scaler_Y[output_var].fit_transform(df_train[[output_var]])
    y_val_scaled[output_var] = scaler_Y[output_var].transform(df_val[[output_var]])
# Define improved kernels
kernels = {
    "rg": C(1.0, (1e-3, 1e2)) * RBF(1.0, (1e-3, 1e2)),       # Standard RBF
    "anis": C(1.0, (1e-3, 1e2)) * RBF(1.0, (1e-3, 1e2)),     # Standard RBF
    "asph": C(1.0, (1e-3, 1e2)) * Matern(1.0, (1e-3, 1e2), nu=1.5),  # Matérn (rougher)
    "acyl": C(1.0, (1e-3, 1e2)) * Matern(1.0, (1e-3, 1e2), nu=2.5)   # Matérn (smoother)
}

# Train separate GPR models for each output variable
gpr_models = {}
y_pred_val = {}

for output_var in ["rg", "asph", "acyl", "anis"]:
    print(f"Training GPR for {output_var}...")

    # Train Gaussian Process Regressor with modified kernels
    gpr = GaussianProcessRegressor(
        kernel=kernels[output_var], 
        n_restarts_optimizer=20,  # Increased for better optimization
        alpha=1e-6, 
        optimizer="fmin_l_bfgs_b"
    )
    gpr.fit(X_train_scaled, y_train_scaled[output_var].ravel())

    # Store trained model
    gpr_models[output_var] = gpr

    # Predict on validation set
    y_pred_scaled = gpr.predict(X_val_scaled)

    # Transform predictions back to original scale
    y_pred_val[output_var] = scaler_Y[output_var].inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()

# Evaluate using MSE
mse_vals = {var: mean_squared_error(df_val[var], y_pred_val[var]) for var in ["rg", "asph", "acyl", "anis"]}
overall_mse = np.mean(list(mse_vals.values()))

print(f"Validation MSE for each output: {mse_vals}")
print(f"Overall Validation MSE: {overall_mse}")